# Conversational AI - aka Chatbot!

In [40]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import gradio as gr

In [41]:
load_dotenv()
gemini_api_key = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=gemini_api_key)

In [42]:
system_prompt = "you are a helpful assistant. In prompt there is conversation between you and me. Take that into context and chat with me"

In [27]:
# gradio accept us to provide function in the below format where there are 2 parameters : new message and the history of previous
#  messages(user and assistant both)

def chat(message, history):
    messages= [ {'role': m['role'], 'content': m['content']} for m in history] if len(history)!=0 else []
    messages.append({'role': 'user', 'content': message})
    model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_prompt)
    print(messages)
    response = model.generate_content(f'{messages}', stream=True)
    
    result = ""
    for chunk in response:
        result += chunk.text
        yield result
    

In [ ]:
gr.ChatInterface(fn=chat, type='messages').launch()

### multi-shot prompting

In [43]:
system_prompt += "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales evemt.'\
Encourage the customer to buy hats if they are unsure what to get."

In [44]:
system_prompt += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [49]:
# we can modify the system message based on the user's message 

def chat(message, history):
    system_message = system_prompt

    if 'belt' in message:
        system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = history + [{"role": "user", "content": message}]

    model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_message)
    print(messages)
    response = model.generate_content(f'{messages}', stream=True)
    
    result = ""
    for chunk in response:
        result += chunk.text
        yield result

In [50]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'content': 'hi'}]
[{'role': 'user', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'Hi there! Welcome to our store.  What can I help you find today? We have a fantastic sale going on right now – hats are 60% off, and most other items are 50% off!', 'options': None}, {'role': 'user', 'content': 'belt?'}]
[{'role': 'user', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'Hi there! Welcome to our store.  What can I help you find today? We have a fantastic sale going on right now – hats are 60% off, and most other items are 50% off!', 'options': None}, {'role': '